In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import cv2
import numpy as np
import os.path
import requests
import statistics
from enum import IntEnum
from PIL import Image, ImageDraw

Some information about the games from the https://www.harmmade.com/vectorracer/#

Here are available surface types:

|Palette index|Type
--- | --- |
|0 | Boundary
|1 | Road
|2 | Grid
|3 | Checkpoint (red)
|4 | Checkpoint (red), intersection with the grid
|5 | Checkpoint (green)
|6 | Checkpoint (green), intersection with the grid
|7 | Start/finish
|8 | Direction sign
|9 | Direction sign, intersection with the grid
|10| Start point boundaries
|11| Start point

* Map contains grid with the step equal to 15
* Tracks must be exactly 794 pixels wide and at least 599 pixels high
* The grid spacing is 15 pixels; the start point is used as an anchor point for the grid

In [ ]:
class SurfaceType(IntEnum):
    BOUNDARY = 0
    EMPTY = 1
    GRID = 2
    FINISH = 7
    START = 11
    TRACK = 20

def download_image(img):
    if os.path.exists(img):
        return
    data = requests.get('https://www.harmmade.com/vectorracer/tracks/{}'.format(img)).content
    with open(img, 'wb') as handler:
        handler.write(data)

class Track:
    def __init__(self, image):
        self._orig = image
        self._start = self._find_start()
        self._canvas = self._orig.copy()
        self._track = self._fill_track()
        self._grid_step = 15

    def save_canvas(self, name):
        self._canvas.save(name)
    
    def _find_start(self):
        xs = []
        ys = []
        for x in range(self._orig.width):
            for y in range(self._orig.height):
                v = self._orig.getpixel((x,y))
                if v == SurfaceType.START:
                    xs.append(x)
                    ys.append(y)
        return statistics.median(xs), statistics.median(ys)

    def _fill_track(self):
        px = np.array(self._orig)
        px[px != SurfaceType.BOUNDARY] = SurfaceType.EMPTY
        cv2.floodFill(px, None, self._start, SurfaceType.TRACK, 0, 1)
        px[px != SurfaceType.TRACK] = SurfaceType.EMPTY
        rv = Image.fromarray(px, mode='P')
        rv.putpalette(self._orig.getpalette())
        return rv
       
    def pt(self, pos):
        return (self._start[0] + self._grid_step * pos[0], self._start[1] - self._grid_step * pos[1])
    
    def is_valid(self, pos):
        p = self.pt(pos)
        return p[0] >= 0 and p[1] >= 0 and p[0] < self._orig.width and p[1] < self._orig.height
      
    def is_on_track(self, pos):
        p = self.pt(pos)
        return self.is_valid(pos) and self._track.getpixel(p) == SurfaceType.TRACK
    
    def is_on_finish(self, pos):
        p = self.pt(pos)
        return self.is_valid(pos) and self._orig.getpixel(p) == SurfaceType.FINISH
    
    def _intersects_finish_back(self, pos0, pos1):
        return pos0[0] >= 0 and pos1[0] < 0 and pos1[1] < 10
    
    def is_on_track_and_finishes(self, pos0, pos1):
        if self._intersects_finish_back(pos0, pos1):
            return False
        if not self.is_on_track(pos0):
            return False
        for p in np.linspace(pos0, pos1, 10):
            if self.is_on_finish(p):
                return True
            if not self.is_on_track(p):
                return False
        return self.is_on_finish(pos1)
    
    def find_finish(self, pos0, pos1):
        for p in np.linspace(pos0, pos1, 100):
            if self.is_on_finish(p):
                return p
        return None        

    def find_finish_param(self, pos0, pos1):
        p = self.find_finish(pos0, pos1)
        if p is None:
            return None
        num = (pos1[0]-pos0[0])*(p[0]-pos0[0]) + (pos1[1]-pos0[1])*(p[1]-pos0[1])
        denom = (pos1[0]-pos0[0])*(pos1[0]-pos0[0]) + (pos1[1]-pos0[1])*(pos1[1]-pos0[1])
        return num / denom
        
    def is_on_track_segment(self, pos0, pos1):
        if self._intersects_finish_back(pos0, pos1):
            return False
        if not self.is_on_track(pos0):
            return False
        for p in np.linspace(pos0, pos1, 10):
            if self.is_on_finish(p):
                return True
            if not self.is_on_track(p):
                return False
        return self.is_on_track(pos1)
    
    def draw(self, pos):
        if self.is_on_track(pos):
            d = 3
            fill = 6
        else:
            d = 10
            fill = 3
        img = ImageDraw.Draw(self._canvas)
        (px, py) = self.pt(pos)
        img.ellipse([(px-d, py-d), (px+d, py+d)], fill=fill, outline=30)
        
    def draw_segment(self, pos0, pos1):
        if self.is_on_track_segment(pos0, pos1):
            fill = 77
            width = 0
        else:
            fill = 3
            width = 3
        p0 = self.pt(pos0)
        p1 = self.pt(pos1)
        img = ImageDraw.Draw(self._canvas)
        img.line([p0, p1], fill=fill, width=width)
        
    def draw_trajectory(self, pts):
        for i in range(1, len(pts)):
            self.draw_segment(pts[i-1], pts[i])
        for pt in pts:
            self.draw(pt)

def get_dst_sqr(pos0, pos1):
    (x0, y0) = pos0
    (x1, y1) = pos1
    return (x1-x0)*(x1-x0)+(y1-y0)*(y1-y0)
    

In [ ]:
download_image('oval.png')
track = Track(Image.open('oval.png'))

In [ ]:
track._track

In [ ]:
def step(state, dd):
    (x, y, dx, dy) = state
    (ddx, ddy) = dd
    return (x + dx + ddx, y + dy + ddy, dx + ddx, dy + ddy)

def step_back(state, dd):
    (x, y, dx, dy) = state
    (ddx, ddy) = dd
    return (x - dx, y - dy, dx - ddx, dy - ddy)
     
def bfs(tr):
    state = (0, 0, 0, 0)
    l = [state]
    idx = 0
    d = {state: 0}
    back = {}
    finishes = []
    found_distance = None
    while idx < len(l):
        el = l[idx]
        dist = d[el]
        idx = idx + 1
        if not found_distance is None and dist >= found_distance:
            break
        for ddx in (-1, 0, 1):
            for ddy in (-1, 0, 1):
                s = step(el, (ddx, ddy))
                if abs(s[2]) > 6 or abs(s[3]) > 6:
                    continue
                if s in d:
                    continue
                if not tr.is_on_track_segment(el, s):
                    continue
                back[s] = (ddx, ddy)
                d[s] = dist + 1
                if tr.is_on_track_and_finishes(el, s):
                    finishes.append(s)
                    found_distance = dist + 1
                else:
                    l.append(s)
    return finishes, back

def trajectory(finish, back):
    tr = [finish]
    while tr[-1] in back:
        last = tr[-1]
        dd = back[last]
        new_last = step_back(last, dd)
        tr.append(new_last)
    return list(reversed(tr))

def trajectory_length(track, tr):
    b = tr[-1]
    a = tr[-2]
    p = round(track.find_finish_param(a, b), 2)
    return len(tr) - 1 + p

def last_speed(tr):
    a = tr[-1]
    return a[2] * a[2] + a[3] * a[3]

In [ ]:
finishes, back = bfs(track)
trajectories = [trajectory(f, back) for f in finishes]
weigthed_trajectories = sorted([(trajectory_length(track, tr), last_speed(tr), tr) for tr in trajectories])
best = weigthed_trajectories[0][2]

In [ ]:
track.draw_trajectory(best)
track.save_canvas('oval-best.png')